In [1]:
from sqlalchemy import create_engine, text
from urllib.parse import urlparse
import os
import polars as pl
from dotenv import load_dotenv
load_dotenv()

True

In [2]:

# Подключение к PostgreSQL
tmpPostgres = urlparse(os.getenv("DATABASE_URL"))

# Синхронный движок для PostgreSQL
engine = create_engine(
    f"postgresql://{tmpPostgres.username}:{tmpPostgres.password}@{tmpPostgres.hostname}{tmpPostgres.path}?sslmode=require",
    echo=True
)

In [3]:
with engine.begin() as conn:
    df = pl.read_database(text("SELECT * FROM warehouse;"), conn, infer_schema_length=10000)
df

2025-03-30 13:05:19,768 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-30 13:05:19,770 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:05:19,865 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-30 13:05:19,866 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:05:19,962 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-30 13:05:19,963 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:05:20,058 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-30 13:05:20,118 INFO sqlalchemy.engine.Engine SELECT * FROM warehouse;
2025-03-30 13:05:20,118 INFO sqlalchemy.engine.Engine [generated in 0.00049s] {}
2025-03-30 13:05:20,264 INFO sqlalchemy.engine.Engine COMMIT


index,name,value,price,comments
i64,str,i64,str,str
0,"""Абсорбер (фильтр угольный)""",10,"""1500""",null
1,"""аморты задние""",10,"""3000""",null
2,"""бампер задний коротыш дорестай…",0,"""3000""",null
3,"""бампер задний коротыш рестайл …",10,"""3000""",null
4,"""бампер передний дорестайл глад…",0,"""7500""",null
…,…,…,…,…
289,"""хабы автоматы""",10,"""6000-9000""",null
290,"""хабы вакуум""",0,"""6000-9000""",null
291,"""хабы механика""",0,"""6000-9000""",null


In [4]:
new_df = df.with_columns(
    (pl.all_horizontal(pl.col('price').str.contains('-'), pl.col('value') == 0)).alias('check_stock_availability')
)
new_df

index,name,value,price,comments,check_stock_availability
i64,str,i64,str,str,bool
0,"""Абсорбер (фильтр угольный)""",10,"""1500""",null,false
1,"""аморты задние""",10,"""3000""",null,false
2,"""бампер задний коротыш дорестай…",0,"""3000""",null,false
3,"""бампер задний коротыш рестайл …",10,"""3000""",null,false
4,"""бампер передний дорестайл глад…",0,"""7500""",null,false
…,…,…,…,…,…
289,"""хабы автоматы""",10,"""6000-9000""",null,false
290,"""хабы вакуум""",0,"""6000-9000""",null,true
291,"""хабы механика""",0,"""6000-9000""",null,true


In [5]:
# Подключение к PostgreSQL
tmpPostgres = urlparse(os.getenv("DATABASE_URL"))

# Синхронный движок для PostgreSQL
engine = create_engine(
    f"postgresql://{tmpPostgres.username}:{tmpPostgres.password}@{tmpPostgres.hostname}{tmpPostgres.path}?sslmode=require",
    echo=True
)
with engine.begin() as conn:
    new_df.write_database("warehouse", conn, if_table_exists='replace')

2025-03-30 13:05:23,234 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-30 13:05:23,236 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:05:23,326 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-30 13:05:23,327 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:05:23,418 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-30 13:05:23,419 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:05:23,509 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-30 13:05:25,148 INFO sqlalchemy.engine.Engine SELECT pg_catalog.pg_class.relname 
FROM pg_catalog.pg_class JOIN pg_catalog.pg_namespace ON pg_catalog.pg_namespace.oid = pg_catalog.pg_class.relnamespace 
WHERE pg_catalog.pg_class.relname = %(table_name)s AND pg_catalog.pg_class.relkind = ANY (ARRAY[%(param_1)s, %(param_2)s, %(param_3)s, %(param_4)s, %(param_5)s]) AND pg_catalog.pg_table_is_visible(pg_catalog.pg_class.oid) AND pg_catalog.pg_namespace.nspname != %(nspname

In [6]:
df = df.with_columns(df['check_stock_availability'].cast(pl.String).replace({'false' : 'уточнять количество НЕ НАДО', "true" : 'уточнять количество НАДО'}))
df

index,name,value,price,check_stock_availability
i64,str,i64,i64,str
1,"""задний бампер автомобиля""",10,3000,"""уточнять количество НЕ НАДО"""
3,"""тормозной барабан колёс""",10,2000,"""уточнять количество НЕ НАДО"""
4,"""бардачок салона""",10,1000,"""уточнять количество НЕ НАДО"""
5,"""бачок гидроусилителя руля""",10,1200,"""уточнять количество НЕ НАДО"""
6,"""бензонасос для калининградског…",0,9000,"""уточнять количество НЕ НАДО"""
…,…,…,…,…
182,"""форсунки топливные""",10,11000,"""уточнять количество НЕ НАДО"""
184,"""электронный блок управления с …",0,12000,"""уточнять количество НЕ НАДО"""
0,"""крышка багажного отделения""",5,13000,"""уточнять количество НЕ НАДО"""


In [7]:
df['check_stock_availability'].value_counts()

check_stock_availability,count
str,u32
"""уточнять количество НЕ НАДО""",138
"""уточнять количество НАДО""",40


In [ ]:

new_column = 'check_stock_availability'
new_df = df.with_columns(
    ((df['value'] == 0) & pl.Series(rand_number)).alias(new_column)
)
new_df

index,name,value,price,check_stock_availability
i64,str,i64,i64,bool
1,"""задний бампер автомобиля""",10,3000,false
3,"""тормозной барабан колёс""",10,2000,false
4,"""бардачок салона""",10,1000,false
5,"""бачок гидроусилителя руля""",10,1200,false
6,"""бензонасос для калининградског…",0,9000,false
…,…,…,…,…
182,"""форсунки топливные""",10,11000,false
184,"""электронный блок управления с …",0,12000,false
0,"""крышка багажного отделения""",5,13000,false


In [28]:
new_df['check_stock_availability'].value_counts()

check_stock_availability,count
bool,u32
true,40
false,138


In [21]:
# Подключение к PostgreSQL
tmpPostgres = urlparse(os.getenv("DATABASE_URL"))

# Синхронный движок для PostgreSQL
engine = create_engine(
    f"postgresql://{tmpPostgres.username}:{tmpPostgres.password}@{tmpPostgres.hostname}{tmpPostgres.path}?sslmode=require",
    echo=True
)
with engine.begin() as conn:
    new_df.write_database("warehouse", conn, if_table_exists='replace')

2025-03-30 13:04:50,933 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2025-03-30 13:04:50,935 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:04:51,025 INFO sqlalchemy.engine.Engine select current_schema()
2025-03-30 13:04:51,027 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:04:51,117 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2025-03-30 13:04:51,118 INFO sqlalchemy.engine.Engine [raw sql] {}
2025-03-30 13:04:51,209 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-03-30 13:04:51,211 INFO sqlalchemy.engine.Engine ROLLBACK


ModuleNotFoundError: writing with 'sqlalchemy' engine currently requires pandas.

Install with: pip install pandas